In this lab you will implement the **skip-gram model with negative sampling (SGNS)** from Lecture&nbsp;1.4, and use it to train word embeddings on the text of the [Simple English Wikipedia](https://simple.wikipedia.org/wiki/Main_Page).

⚠️ The dataset for this lab contains 18M tokens. This is very little as far as word embedding datasets are concerned – for example, the original word2vec model was pre-trained on 100B tokens. In spite of this, you will need to think about efficiency when processing the data and training your models. In particular, wherever possible you should use iterators rather than lists, and vectorize operations (using [NumPy](https://numpy.org) or [PyTorch](https://pytorch.org)) as much as possible.

# L1: Word representations

## Load the data

The data for this lab comes as a bz2-compressed plain text file. It consists of 1,163,769 sentences, with one sentence per line and tokens separated by spaces. The cell below contains a wrapper class `SimpleWikiDataset` that can be used to iterate over the sentences (lines) in the text file. On the Python side of things, each sentence is represented as a list of tokens (strings).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import bz2

class SimpleWikiDataset():
    
    def __init__(self, max_sentences=None):
        self.max_sentences = max_sentences
    
    def __iter__(self):
        with bz2.open('/content/drive/MyDrive/TDDE09/labs/l1/simplewiki.txt.bz2', 'rt', encoding='utf-8') as sentences: # unzip manually if it is not working
            for i, sentence in enumerate(sentences):
                if self.max_sentences and i >= self.max_sentences:
                    break
                yield sentence.split()

Using this class, we define two variants of the dataset: the full dataset and a minimal version with the first 1% of the sentences in the full dataset. The latter will be useful to test code without running it on the full dataset.

In [ ]:
# Dataset with all sentences (N = 1,163,769)
full_dataset = SimpleWikiDataset()

# Minimal dataset
mini_dataset = SimpleWikiDataset(max_sentences=11638)

CURR_DATASET = mini_dataset

The next code cell defines a generator function that allows you to iterate over all tokens in a dataset:

In [ ]:
def tokens(sentences):
    for sentence in sentences:
        for token in sentence:
            yield token

To illustrate how to use this function, here is code that prints the number of tokens in the full dataset:

In [ ]:
print(sum(1 for t in tokens(full_dataset)))

17594885


## Problem 1: Build the vocabulary and frequency table

Your first task is to construct the embedding **vocabulary** – the set of unique words that will receive an embedding. Because you will eventually need to map words to vector dimensions, you will represent the vocabulary as a dictionary that maps words (strings) to a contiguous range of integers.

Along with the vocabulary, you will also construct the **frequency table**, that is, the table that holds the absolute frequencies (counts) in the data, for all words in your vocabulary. This will simply be an array of integers, indexed by the word ids in the vocabulary.

To construct the vocabulary and the frequency table, complete the skeleton code in the cell below:

In [ ]:
import numpy as np

def make_vocab_and_counts(sentences, min_count=5):
    vocab = {}        # The vocabulary to be returned.
    counts = {}       # For simple count of all words without index.
    freq_table = []   # The frequenct table to be returned. Converted to np array later.

    for sentence in sentences:
      for token in sentence:
        if token not in counts:
          counts[token] = 1
        else:
          counts[token] += 1

    for sentence in sentences: 
      for token in sentence:
        idx = len(vocab)
        if token not in vocab and counts[token] >= min_count:
          vocab[token] = idx
          freq_table.append(counts[token])

    return vocab, np.array(freq_table)

Your code should comply with the following specification:

**make_vocab_and_counts** (*sentences*, *min_count* = 5)

> Reads from an iterable of *sentences* (lists of string tokens) and returns a pair *vocab*, *counts* where *vocab* is a dictionary representing the vocabulary and *counts* is a 1D-[ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) with the absolute frequencies (counts) of the words in the vocabulary. The dictionary *vocab* maps words to a contiguous range of integers starting at&nbsp;0. In the *counts* array, the entry at index $i$ is the count of that word in *vocab* which maps to $i$. Words that occur less than *min_count* times are excluded from the vocabulary.

### 🤞 Test your code

To test your code, print the sizes of the vocabularies constructed from the two datasets, as well as the count totals. The correct vocabulary size for the minimal dataset is 3,231; for the full dataset, the correct vocabulary size is 73,339. The correct totals are 155,818 for the minimal dataset and 17,297,355 for the full dataset.

In [ ]:
# TODO: Test your code here
vocab, freq_table = make_vocab_and_counts(CURR_DATASET)
print(vocab['usually'])
print(len(freq_table))
print(len(vocab))

print(vocab)
print(sum(freq_table)) # Correct!! :)

3
3231
3231
{'their': 0, 'name': 1, 'is': 2, 'usually': 3, 'said': 4, 'as': 5, 'it': 6, 'can': 7, 'not': 8, 'be': 9, 'searched': 10, 'for': 11, 'on': 12, 'written': 13, 'like': 14, 'that': 15, 'they': 16, 'started': 17, 'in': 18, 'california': 19, 'have': 20, 'released': 21, 'five': 22, 'albums': 23, 'fifth': 24, 'album': 25, 'was': 26, 'april': 27, 'gold': 28, 'standard': 29, 'or': 30, 'how': 31, 'i': 32, 'to': 33, 'stop': 34, 'and': 35, 'love': 36, 'the': 37, 'episode': 38, 'of': 39, 'season': 40, 'first': 41, 'shown': 42, 'television': 43, 'december': 44, 'springfield': 45, 'its': 46, 'economy': 47, 'burns': 48, 'a': 49, 'where': 50, 'bart': 51, 'his': 52, 'appearance': 53, 'state': 54, 'howard': 55, 'also': 56, 'known': 57, 'yellow': 58, 'spotted': 59, 'tree': 60, 'frog': 61, 'new': 62, 'england': 63, 'bell': 64, 'from': 65, 'eastern': 66, 'australia': 67, 'people': 68, 'thought': 69, 'these': 70, 'frogs': 71, 'were': 72, 'but': 73, 'then': 74, 'scientists': 75, 'found': 76, 'some'

## Problem 2: Preprocess the data

Your next task is to preprocess the training data. This involves the following:

* Discard words that are not in the vocabulary
* Map each word to its vocabulary id
* Randomly discard words according to the subsampling strategy covered in Lecture&nbsp;1.4
* Discard sentences that have become empty

As a reminder, the subsampling strategy involves discarding tokens $w$ with probability

$$
P(w) = \max (0, 1-\sqrt{tN/\#(w)})
$$

where $\#(w)$ is the count of $w$, $N$ is the total number of counts, and $t$ is the chosen threshold (default value: 0.001).

The cell below contains skeleton code for a generator function `preprocess`:

In [ ]:
import torch
import math

def discard(N, dist, w, t = 0.001):
  random_points = torch.rand(1) #* dist[-1]
  torch.searchsorted(dist, random_points)

  pw = max(0, 1 - math.sqrt(t*N / w))
  return pw > random_points

def preprocess(vocab, counts, sentences, threshold=0.001):
    # TODO: Replace the next line with your own code
    N = sum(counts)
    cumulative_sums = torch.cumsum(torch.from_numpy(counts), dim=0)

    for sentence in sentences:
      list_of_ids = []
      for word in sentence:
        if word in vocab and not discard(N, cumulative_sums, counts[vocab[word]], threshold):          
          list_of_ids.append(vocab[word])

      if list_of_ids:
        yield list_of_ids
# Worst case for discarding randomly is that the whole sentence is gone, need to be implemented?

Extend this skeleton code into a function that implements the preprocessing. Your code should comply with the following specification:

**preprocess** (*vocab*, *counts*, *sentences*, *threshold* = 0.001)

> Reads from an iterable of *sentences* (lists of string tokens) and yields the preprocessed sentences as non-empty lists of word ids (integers). Words not in *vocab* are discarded. The remaining words are randomly discarded according to the subsampling strategy with the given *threshold*. In the non-empty sentences, each token is replaced by its id in the vocabulary.

**⚠️ Please observe** that your function should *yield* the preprocessed sentences, not return a list with all of them. That is, we ask you to write a *generator function*. If you have not worked with generators and iterators before, now is a good time to read up on them. [More information about generators](https://wiki.python.org/moin/Generators)

### 🤞 Test your code

Test your code by comparing the total number of tokens in the preprocessed version of each dataset with the corresponding number for the original data. The former should be ca. 59% of the latter for the minimal dataset, and ca. 69% for the full dataset. The exact percentage will vary slightly because of the randomness in the sampling. You may want to repeat your computation several times.

In [ ]:
processed = [sentence_list for sentence_list in preprocess(vocab, freq_table, CURR_DATASET)]
print(sum(1 for t in tokens(processed)))
print(sum(1 for t in tokens(CURR_DATASET)))

print(sum(1 for t in tokens(processed)) / sum(1 for t in tokens(CURR_DATASET)))


101451
170815
0.5939232503000322


## Problem 3: Generate the training examples

Your next task is to translate the preprocessed sentences into training examples for the skip-gram model: both *positive examples* (target word–context word pairs actually observed in the data) and *negative examples* (pairs randomly sampled from a noise distribution).

**⚠️ We expect that solving this problem will take you the longest time in this lab.**

### General strategy

The general plan for solving this problem is to implement a generator function that traverses the preprocessed sentences, at each position of the text samples a window, and then extracts all positive examples from it. For each positive example, the function also generates $k$ negative examples, where $k$ is a hyperparameter. Finally, all examples (positive and negative) are combined into the tensor representation described below.

### Representation

How should you represent a batch of training examples? Writing $B$ for the batch size, the obvious choice would be to represent the inputs as a matrix of shape $[B, 2]$ and the output labels (positive/negative) as a vector of length $B$. This representation would be quite wasteful on the input side, however, as each target word (index) from a positive example would have to be repeated in all negative samples. For example ($k=3$):

Here you will use a different representation: First, instead of a single input batch, there will be a *pair* of input batches – a vector for the target words and a matrix for the context words. If the target word vector has length $B$, the context word matrix has shape $[B, 1+k]$. The $i$th element of the target word vector is the target word for *all* context words in the $i$th row of the context word matrix: the first column of that row comes from a positive example, the remaining columns come from the $k$ negative samples. Accordingly, the batch with the output labels will be a matrix of the same shape as the context word matrix, with its first column set to&nbsp;1 and its remaining columns set to&nbsp;0. Corresponding to the example above:

For the present problem, you will only be concerned with the two input batches; the output batch will be constructed in the training procedure. In fact, for a fixed batch size $B$, that batch is always exactly the same, so you will only have to build it once.

### Negative sampling

Recall from Lecture&nbsp;1.4 that the probability of a word $c$ to be selected as the context word in a negative sample is proportional to its exponentiated count $\#(c)^\alpha$, where $\alpha$ is a hyperparameter (default value: 0.75).

To implement negative sampling from this distribution, you can follow a standard recipe: Start by pre-computing an array containing the *cumulative sums* of the exponentiated counts. Then, generate a random cumulative count $n$, and find that index in the pre-computed array at which $n$ should be inserted to keep the array sorted. That index identifies the sampled context word.

All operations in this recipe can be implemented efficiently in PyTorch; the relevant functions are [`torch.cumsum`](https://pytorch.org/docs/stable/generated/torch.cumsum.html) and [`torch.searchsorted`](https://pytorch.org/docs/stable/generated/torch.searchsorted.html). For optimal efficiency, you should sample all $B \times k$ negative examples in a batch at once.

Here is skeleton code for this problem:

In [ ]:
from torch import LongTensor
# Not used
def get_target_words(sentence, window_size):
  target_words = []
  tgt_idx = [idx for idx, id in enumerate(sentence) if idx >= math.floor(window_size / 2) and idx <= len(sentence) - math.floor(window_size / 2)]
  return tgt_idx

def get_context_words_and_negative_samples(sentence, target_idx, window_size, cum_sum, k):
  list_of_ids = []

  positive_c_words = window_size - 1
  fun = int(positive_c_words / 2)
  component_1 = torch.LongTensor(target_idx, positive_c_words)
  component_2 = torch.LongTensor(target_idx, positive_c_words, 1 + k)
  output_label = torch.LongTensor(target_idx, positive_c_words, 1 + k)
  output_label[:, :, 0] = 1
  output_label[:, :, 1:] = 0

  c_words_temp = torch.LongTensor(positive_c_words)

  # print("sentence: ", sentence)
  # print("window_size: ", window_size)
  # print("positive context words: ", positive_c_woslrds)
  # print()

  for tgt_word_idx in range(0, target_idx):
    #for i in range(0,2):
      if tgt_word_idx == 0:
        c_words_temp[0:positive_c_words + 1] = torch.LongTensor(sentence[1: positive_c_words + 1])

      elif tgt_word_idx < positive_c_words / 2:
        c_words_temp[0: tgt_word_idx] = torch.LongTensor(sentence[0:tgt_word_idx])
        c_words_temp[tgt_word_idx + 1: positive_c_words - tgt_word_idx] = torch.LongTensor(sentence[tgt_word_idx + 1: positive_c_words - tgt_word_idx])

      elif tgt_word_idx == target_idx - 1:
        c_words_temp[0:positive_c_words] = torch.LongTensor(sentence[tgt_word_idx - positive_c_words: tgt_word_idx])

      elif tgt_word_idx > target_idx - positive_c_words / 2:
        c_words_temp[0:fun + 1] = torch.LongTensor(sentence[tgt_word_idx:target_idx])
        c_words_temp[fun:positive_c_words] = torch.LongTensor(sentence[tgt_word_idx - positive_c_words + target_idx - tgt_word_idx : tgt_word_idx])
      
      else:
        c_words_temp[0 : fun + 1] = torch.LongTensor(sentence[tgt_word_idx - fun : tgt_word_idx + 1])
        c_words_temp[fun : positive_c_words] = torch.LongTensor(sentence[tgt_word_idx + 1 : tgt_word_idx + fun + 1])

   
      component_2[tgt_word_idx, :, 0] = c_words_temp

      for i in range(positive_c_words):
        random_points = torch.rand(k) * cum_sum[-1]
        neg_samples = torch.searchsorted(cum_sum, random_points)

        component_1[tgt_word_idx, i] = sentence[tgt_word_idx]
        component_2[tgt_word_idx, i, 1:] = neg_samples

  #print(component_1)
  #print(component_2)  
  #print(output_label)  

  return component_1, component_2, output_label

def training_examples(vocab, counts, sentences, window=5, num_ns=5, batch_size=1<<19, ns_exponent=0.75):
    # TODO: Replace the next line with your own code
    cumulative_sums = torch.cumsum(torch.from_numpy(counts**ns_exponent), dim=0)

    preprocessed = [sentence for sentence in preprocess(vocab, freq_table, sentences)]
    print("Filtered_sentence_length: ", len(preprocessed))

    small = []
    big = []
    ctr=0
    for sentence in preprocessed: # 1
      if len(sentence) >= 5: # Fixed with slice
        target_word_idxs = len(sentence)

        sampled_window_size = int(np.random.uniform(2, window))
        if sampled_window_size % 2 == 0:
          sampled_window_size += 1

        tgt_word_vec, cont_word_mat, output_label = get_context_words_and_negative_samples(sentence, target_word_idxs, sampled_window_size, cumulative_sums, num_ns)
        small.append(tgt_word_vec) # = x.append?
        big.append(cont_word_mat)


    # Flatten tensors as they were sentence paired from our function
    small_flattened = [tens.flatten() for tens in small]
    big_flattened = [tens.flatten(start_dim=0, end_dim=1) for tens in big]

    # Count total number of examples
    total_tgt_vec_examples = 0
    for tens in small_flattened:
      total_tgt_vec_examples += tens.size(dim=0) 
    total_cont_word_examples = 0
    for tens in big_flattened:
      total_cont_word_examples += tens.size(dim=0)

    # Create tensors to store total examples
    all_tgt_vec_examples = torch.zeros(total_tgt_vec_examples).long()
    all_cont_word_examples = torch.zeros(total_cont_word_examples, 1 + num_ns).long()

    # Fill tensors with all examples
    curr_size = 0
    for tens in small_flattened:
      tens_size = tens.size(dim=0)
      all_tgt_vec_examples[curr_size: curr_size + tens_size] = tens[:]
      curr_size += tens_size

    curr_size = 0
    for tens in big_flattened:
      tens_size = tens.size(dim=0)
      all_cont_word_examples[curr_size: curr_size + tens_size] = tens[:]
      curr_size += tens_size

    # Yield as many batch size pairs as we can
    n_pairs = math.floor(total_cont_word_examples / batch_size)
    
    # Kommer ej in hit
    for curr_b_size in range(n_pairs):
      comp1 = torch.zeros(batch_size).long() # Sätter att comp1 ska vara batch size stor
                                             # Är ej det för de andra, den är bara "så stor
                                             # den kan vara"
      comp2 = torch.zeros(batch_size, 1 + num_ns).long()

      comp1[0: batch_size] = all_tgt_vec_examples[curr_b_size * batch_size: (curr_b_size + 1) * batch_size]
      comp2[0: batch_size] = all_cont_word_examples[curr_b_size * batch_size: (curr_b_size + 1) * batch_size]
      
      yield comp1, comp2
    
    # Om vi inte kommer till for loop ska vi yielda det vi har
    # yield torch.tensor(comp1), torch.cat((torch.tensor(comp2), ns[:len(torch.tensor(comp1))]), 1)
    
    #yield torch.zeros(batch_size).long(), torch.zeros(batch_size, 1 + num_ns).long()

Your code should comply with the following specification:

**training_examples** (*vocab*, *counts*, *sentences*, *window* = 5, *num_ns* = 5, *batch_size* = 524,288, *ns_exponent*=0.75)

> Reads from an iterable of *sentences* (lists of string tokens), preprocesses them using the function implemented in Problem&nbsp;2, and then yields pairs of input batches for gradient-based training, represented as described above. Each batch contains *batch_size* positive examples. The parameter *window* specifies the maximal distance between a target word and a context word in a positive example; the actual window size around any given target word is sampled uniformly at random. The parameter *num_ns* specifies the number of negative samples per positive sample. The parameter *ns_exponent* specifies the exponent in the negative sampling (called $\alpha$ above).

### 🤞 Test your code

To test your code, compare the total number of positive samples (across all batches) to the total number of tokens in the (un-preprocessed) minimal dataset. The ratio between these two values should be ca. 2.64. If you can spare the time, you can make the same comparison on the full dataset; here, the expected ratio is 3.25. As before, the numbers may vary slightly because of randomness, so you may want to run the comparison more than once.

In [ ]:
# TODO: Test your code here
original_length = sum([len(token) for token in CURR_DATASET])

counter = 0
batches = 0 # Ska vara en batch
foo = 0
w,c = [],[]
for comp1, comp2 in training_examples(vocab, freq_table, list(CURR_DATASET)):
  print()


Filtered_sentence_length:  11625


In [ ]:
tst_counts = np.array([14507, 5014, 4602, 4529, 4000, 3219, 3010, 2958, 2225, 1271])
tst_sentence = [["the","cat", "sat", "on", "the", "mat", "cat", "mat"], ["cat","cat", "cat", "on", "the", "mat", "mat", "mat"]]
tst_vocab = {"hello": 8, "world":1, "hej": 2, "the": 3, "cat": 4, "sat": 5, "on": 6, "mat": 7}
# for comp1, comp2 in training_examples(tst_vocab, tst_counts, tst_sentence, 5, 5, 4):
#   print("\n")
#   print(comp1)
#   print()
#   print(comp2)


In [ ]:
sum(1 for t in tokens(CURR_DATASET))

170815

## Problem 4: Implement the model

Now it is time to implement the skip-gram model as such. The cell below contains skeleton code for this. As you will recall from Lecture&nbsp;1.4, the core of the implementation is formed by two embedding layers: one for the target word representations, and one for the context word representations. Your task is to implement the missing `forward()` method.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SGNSModel(nn.Module):
    
    def __init__(self, vocab, embedding_dim):
        super().__init__()
        self.vocab = vocab
        self.w = nn.Embedding(len(vocab), embedding_dim)
        self.c = nn.Embedding(len(vocab), embedding_dim)
    
    def forward(self, w, c):
        # TODO: Replace the next line with your own code
        return torch.zeros_like(c, dtype=torch.float, requires_grad=True) # can be one line of code 

Your implementation of the `forward()` method should comply with the following specification:

**forward** (*self*, *w*, *c*)

> The input to this methods is a tensor *w* with target words of shape $[B]$ and a tensor *c* with context words of shape $[B, 1+k]$, where $B$ is the batch size and $k$ is the number of negative samples. The two tensors are structured as explained for Problem&nbsp;3. The output of the method is a tensor $D$ of shape $[B, k+1]$ where entry $D_{ij}$ is the dot product between the embedding vector for the $i$th target word and the embedding vector for the context word in row $i$, column $j$.

**💡 Hint:** To compute a dot product $x^\top y$, you can first compute the Hadamard product $z = x \odot y$ and then sum up the elements of $z$.

### 🤞 Test your code

Test your code by creating an instance of the model, and check that `forward` returns the expected result on random input tensors *w* and *c*. To help you, the following function will return a random example from the first 100 examples produced by `training_examples`.

In [ ]:
# TODO: Test your code here

In [ ]:
import numpy as np

def random_example(vocab, counts, sentences):
    skip = np.random.randint(100)
    for i, example in enumerate(training_examples(vocab, counts, sentences, num_ns=1, batch_size=5)):
        if i >= skip:
            break
    return example

## Problem 5: Train the model

Once you have a working model, it is time to train it. The training loop for the skip-gram model will be very similar to the prototypical training loop that you saw in Lecture&nbsp;0.6, with two things to note:

First, instead of categorical cross entropy, you will use binary cross entropy. Just like the standard implementation of the softmax classifier, the skip-gram model does not include a final non-linearity, so you should use [`binary_cross_entropy_with_logits()`](https://pytorch.org/docs/1.9.1/generated/torch.nn.functional.binary_cross_entropy_with_logits.html).

The second thing to note is that you will have to create the tensor with the output labels, as explained already in Problem&nbsp;3. This should be a matrix of size $[B, 1+k]$ whose first column contains $1$s and whose remaining columns contains $0$s.

Here is skeleton code for the training loop, including default values for the most important hyperparameters:

In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim

def train(sentences, embedding_dim=50, window=5, num_ns=5, batch_size=1<<19, n_epochs=1, lr=1e-1):
    # Create the vocabulary and the counts
    vocab, counts = make_vocab_and_counts(sentences)
    
    # Initialize the model
    model = SGNSModel(vocab, embedding_dim)
    
    # Initialize the optimizer. Here we use Adam rather than plain SGD
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # TODO: Add your code here
    
    return model

To show you how `train` is meant to be used, the code in the next cell trains a model on the minimal dataset.

In [ ]:
model = train(mini_dataset, n_epochs=1)

### 🤞 Test your code

Test your implementation of the training loop by training a model on the minimal dataset. This should only take a few seconds. You will not get useful word vectors, but you will be able to see whether your code runs without errors.

Once you have passed this test, you can train a model on the full dataset. Print the loss to check that the model is actually learning; if the loss is not decreasing, try to find the problem before wasting time (and energy) on useless training.

Training on the full dataset will take some time – on a CPU, you should expect 10–40 minutes per epoch, depending on hardware. To give you some guidance: The total number of positive examples is approximately 58M, and the batch size is chosen so that each batch contains roughly 10% of these examples. To speed things up, you can train using a GPU; our reference implementation runs in less than 2 minutes per epoch on [Colab](http://colab.research.google.com).

In [ ]:
# TODO: Train your model on the full dataset here

## Problem 6: Analyse the embeddings (reflection)

Now that you have a trained model, you will probably be curious to see what it has learned. You can inspect your embeddings using the [Embedding Projector](http://projector.tensorflow.org). To that end, click on the ‘Load’ button, which will open up a dialogue with instructions for how to upload embeddings from your computer.

You will need to upload two tab-separated files. To create them, you can use the following code:

In [ ]:
def save_model(model):
    # Extract the embedding vectors as a NumPy array
    embeddings = model.w.weight.detach().numpy()
    
    # Create the word–vector pairs
    items = sorted((i, w) for w, i in model.vocab.items())
    items = [(w, e) for (i, w), e in zip(items, embeddings)]
    
    # Write the embeddings and the word labels to files
    with open('vectors.tsv', 'wt') as fp1, open('metadata.tsv', 'wt') as fp2:
        for w, e in items:
            print('\t'.join('{:.5f}'.format(x) for x in e), file=fp1)
            print(w, file=fp2)

Take some time to explore the embedding space. In particular, inspect the local neighbourhoods of words that you are curious about, say the 10 closest neighbours. Document your exploration in a short reflection piece (ca. 150&nbsp;words). Respond to the following prompts:

* Which words did you try? Which results did you get? Did you do anything else than inspecting local neighbourhoods?
* Based on what you know about word embeddings, did you expect your results? How do you explain them?
* What did you learn? How, exactly, did you learn it? Why does this learning matter?

*TODO: Enter your text here*

👍 Well done!